Получилась точность в 94% на датасете на 80% процентов состоящем из текста, при выбраном пороге - Precision=80%, Recall=95%, можно значительно улучшить увелечением размера датасета, улучшеним препроцессинга, и усложнением модели.

Текущие решениеи доминировалась экономией места на диске и времени (между ними тоже можно выбирать, пред-распаковка датасета с кодом ускоряет загрузку, но значительно увеличивает занимаемое место)

На предложенной питоновской строчке не срабатывает (но всё-равно выдаёт вероятность на несколько порядков больше чем для предложенной текстовой строчки), так-как обучен на джаве. На джавовских строчках работает.

In [1]:
from torch.utils.data import Dataset, ConcatDataset, dataloader
import json
import nltk
import re
import glob
import numpy as np
import os
import itertools
import tarfile
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import train_test_split

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
ss = nltk.SnowballStemmer(language = 'english')

In [5]:
file_list_chache_meta = None
file_list_cache = None

In [6]:
def get_file_list(path, seed):
    global file_list_cache, file_list_chache_meta

    if file_list_chache_meta == (path, seed):
        return file_list_cache
    
    with tarfile.open(path, 'r:gz') as tarf:
        file_list_cache = tarf.getnames()
    # Preserve state so that seed only affects local random
    random_state = np.random.get_state()
    np.random.seed(seed)
    np.random.shuffle(file_list_cache)
    np.random.set_state(random_state)
    file_list_chache_meta = (path, seed)
    return file_list_cache

In [70]:
def preprocess_sentence(s):
    s = s.lower()
    s = re.sub("s+", " ", s)
    s = re.sub("[^-9A-Za-z ]", " " , s)
    s = nltk.tokenize.word_tokenize(s)
    s = [ss.stem(w) for w in s]
    return s

In [8]:
# https://www.reddit.com/r/datasets/comments/1uyd0t/200000_jeopardy_questions_in_a_json_file/
class JeopardyDataset(Dataset):

    def __init__(self, path='./JEOPARDY_QUESTIONS1.json'):
        with open(path) as f:
            data = json.load(f)
        self.texts = [preprocess_sentence(x['question']) for x in data]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, i):
        return self.texts[i], 'text'

In [9]:
# https://groups.inf.ed.ac.uk/cup/javaGithub/
!wget https://groups.inf.ed.ac.uk/cup/javaGithub/java_projects.tar.gz

--2022-03-16 14:29:09--  https://groups.inf.ed.ac.uk/cup/javaGithub/java_projects.tar.gz
Resolving groups.inf.ed.ac.uk (groups.inf.ed.ac.uk)... 129.215.202.26
Connecting to groups.inf.ed.ac.uk (groups.inf.ed.ac.uk)|129.215.202.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1925914209 (1.8G) [application/gzip]
Saving to: ‘java_projects.tar.gz’

java_projects.tar.g 100%[===================>]   1.79G  11.8MB/s    in 2m 48s  

2022-03-16 14:31:58 (11.0 MB/s) - ‘java_projects.tar.gz’ saved [1925914209/1925914209]



In [10]:
class JavaDataset(Dataset):

    def __init__(self, path='./java_projects.tar.gz', seed=6741, total=200000): 
        files = get_file_list(path, seed)
        
        with tarfile.open(path, 'r:gz') as tarf:
            self.texts = []
            for fn in files:
                if not fn.endswith('.java'):
                    continue
                f = tarf.extractfile(fn)
                if f is None:
                    continue
                try:
                    ln = f.readlines()
                    for l in ln:
                        if len(l) > 20:
                            self.texts.append(preprocess_sentence(l.decode('utf8')))
                except Exception as e:
                    pass
                print(f'{len(self.texts)}/{total}')
                if total is not None and len(self.texts) > total:
                    break
        
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, i):
        return self.texts[i], 'code'

In [76]:
stemmed_texts = [[ss.stem(w) for w in s] for s in jd.texts]

In [71]:
d = JeopardyDataset()

In [15]:
jd = JavaDataset(total=(2/8) * 200000)

61/50000.0
77/50000.0
129/50000.0
137/50000.0
222/50000.0
255/50000.0
389/50000.0
496/50000.0
521/50000.0
556/50000.0
609/50000.0
627/50000.0
702/50000.0
1270/50000.0
1346/50000.0
1364/50000.0
1390/50000.0
1440/50000.0
1526/50000.0
1598/50000.0
1672/50000.0
1728/50000.0
1769/50000.0
1785/50000.0
1804/50000.0
2746/50000.0
2764/50000.0
3078/50000.0
3092/50000.0
3129/50000.0
3192/50000.0
3205/50000.0
4098/50000.0
4144/50000.0
4236/50000.0
4255/50000.0
4301/50000.0
4327/50000.0
4380/50000.0
4516/50000.0
4626/50000.0
4666/50000.0
4716/50000.0
4885/50000.0
4951/50000.0
5005/50000.0
5018/50000.0
5038/50000.0
5084/50000.0
5158/50000.0
5172/50000.0
5259/50000.0
5311/50000.0
5524/50000.0
5536/50000.0
5625/50000.0
5661/50000.0
5667/50000.0
5686/50000.0
5712/50000.0
5733/50000.0
5748/50000.0
5772/50000.0
5775/50000.0
5828/50000.0
5857/50000.0
5875/50000.0
5934/50000.0
5987/50000.0
5998/50000.0
6011/50000.0
6014/50000.0
6094/50000.0
6206/50000.0
6206/50000.0
6366/50000.0
6465/50000.0
6484/50000.0
6

In [82]:
dictionary = {w: i + 1 for i, w in enumerate({w for x in itertools.chain(d.texts, jd.texts) for w in x})}

In [83]:
def encode(s, n=30):
    return [dictionary.get(w, 0) for w in s[:n]] + [0] * (n - len(s))

In [84]:
X = np.array([encode(s) for s in itertools.chain(d.texts, jd.texts)])
y = np.array([0] * len(d) + [1] * len(jd))

In [85]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3)

In [86]:
np.mean(y_train), np.mean(y_valid)

(0.18709259635865436, 0.18788476667374285)

In [87]:
classifier = CategoricalNB(min_categories=len(dictionary) + 1)

In [88]:
classifier.fit(X_train, y_train)

CategoricalNB(min_categories=76113)

In [99]:
cutoff = np.percentile(classifier.predict_proba(X_train[y_train == 0])[:, 1], 95)
print(cutoff)

1.49640073123053e-05


In [100]:
np.mean((classifier.predict_proba(X_valid)[:, 1] > cutoff) == y_valid)

0.9438568449444937

In [101]:
def pred(mask):
    return classifier.predict_proba(X_valid[mask])[:, 1] > cutoff

In [102]:
tp = np.sum(pred(y_valid == 1) == 1)
fp = np.sum(pred(y_valid == 0) == 1)
tn = np.sum(pred(y_valid == 0) == 0)
fn = np.sum(pred(y_valid == 1) == 0)

In [103]:
print(f"Precision: {round(tp / (tp + fp), 2)}")
print(f"Recall: {round(tp / (tp + fn), 2)}")

Precision: 0.8
Recall: 0.93


In [94]:
def predict_proba(s):
    return classifier.predict_proba([encode(preprocess_sentence(s))])[0, 1] 

def predict(s):
    return predict_proba(s) > cutoff

In [106]:
predict('events.dispatch(new ReadyEvent(this));')

True

In [107]:
predict('If the page range is larger than a given size, the whole range is not')

False

In [109]:
predict_proba('yield from range(number + 1, number + on_each_side + 1)')

4.374171344004736e-08

In [108]:
predict_proba('If the page range is larger than a given size, the whole range is not')

3.919099250354157e-13